## Cars in all RUSSIA
- a good guide on scraping data with Beautiful SOup https://www.youtube.com/watch?v=eN_3d4JrL_w&t=201s
- Data was scraped from avito, Region - entire Russia, any car brand, price 450 000 -1m
- Date November 2021

In [1]:
import csv
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import requests

from time import sleep

# PROTOTYPE

## get url

In [2]:
def get_url (page):
    ## Intitial settings on avito.ru , cars, ALL RUSSIA 450000 1m, except damaged
    templ ='https://www.avito.ru/rossiya/avtomobili?f=ASgBAgECAUTyCrCKAQFFxpoMHHsiZnJvbSI6NDUwMDAwLCJ0byI6MTAwMDAwMH0&p={}'
    url = templ.format(page) 

    return url

In [3]:
url = get_url (0)

In [4]:
url

'https://www.avito.ru/rossiya/avtomobili?f=ASgBAgECAUTyCrCKAQFFxpoMHHsiZnJvbSI6NDUwMDAwLCJ0byI6MTAwMDAwMH0&p=0'

## Extract raw html

In [5]:
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
r = requests.get(url, headers)
soup = bs(r.content, 'html.parser')

In [6]:
## Resposne 200 is good
r

<Response [200]>

In [7]:
cars = soup.find_all('div', {'data-marker': 'item'})

In [8]:
len(cars)

56

In [11]:
## Example of a bad response
rr = requests.get('http://httpbin.org/status/404')
rr

<Response [404]>

## Prototype for a single record

In [12]:
car = cars[2]

In [13]:
car_brand = car.find('div', class_ = 'iva-item-titleStep-_CxvN').text
car_brand

'Ford Focus, 2012'

In [14]:
# \xa is "NO-BREAK SPACE".
car_price = car.find('span', class_ = 'price-price-BQkOZ').text.strip()
car_price =  car_price.replace(u'\xa0', ' ')
car_price

'950 000 ₽'

In [15]:
car_url = car.div.a.get('href')
car_url

'/moskva/avtomobili/ford_focus_2012_2290444814'

In [116]:
car_details = car.find('div', class_ = 'iva-item-text-_s_vh text-text-LurtD text-size-s-BxGpL').text.strip()
car_details

'140 000 км, 1.9 AMT (105 л.с.), универсал, передний, дизель'

In [117]:
car_description = car.find('div', class_ = 'iva-item-descriptionStep-QGE8Y').text.strip()
car_description

'Выгодные предложения от автосалона «Голд Авто»: \n\n1. Кредит от 4,9% годовых на все марки авто, за исключением: на марку Lada от 1%*.\n\n2. Государственные программы кредитования «Первый автомобиль» и «Семейный автомобиль»*.\n\n3. Наслаждайтесь новым авто в полной мере! При покупке авто в кредит оплатим первые два месяца за Вас*.\n\n4. Trade-IN с выгодой до 100 000 руб. \n\n5. Всем покупателям комплект зимних шин в подарок*.\n\n* Банк-партнер: Пао Банк Втб. Генеральная лицензия ЦБ РФ № 1000 от 08.07.2015 г. Ставка по кредиту зависит от выбранной марки автомобиля: на автомобили Lada 1%, остальные марки 4,9%. Доступная сумма кредита от 30 000 до 3 000 000 руб. Первоначальный взнос по кредиту до 60%. Срок кредита от 12 мес. до 7 лет. \n\nСумма ежемесячного платежа для всех марок авто, кроме Лада, от 423 руб. до 256 685 руб., без комиссий. Для автомобилей марки Лада сумма ежемесячного платежа от 370 руб. до 251 356,23 руб., без комиссий. \n\n* Количество подарков ограничено. За дополните

In [118]:
car_location = car.find('div', class_ = 'geo-root-H3eWU iva-item-geo-g3iIJ').text.strip()
car_location

'Челябинская область, Челябинск'

In [119]:
ad_date = car.find('div', 
    class_ = 'date-text-VwmJG text-text-LurtD text-size-s-BxGpL text-color-noaccent-P1Rfs').text.strip()
ad_date

'Несколько секунд назад'

In [121]:
car_dealer = car.find('div', 
    class_ = 'iva-item-text-_s_vh iva-item-hideWide-oLeJu text-text-LurtD text-size-s-BxGpL').text.strip()
car_dealer

'Голд Авто'

In [100]:
today = datetime.today().strftime('%Y-%m-%d')

# GENERALIZE WITH A FUNCTION
- sleep, essential , otherwise get blocked

In [5]:
def get_url (page):
    ## cars, ALL RUSSIA 450000 1m, except damaged
    templ ='https://www.avito.ru/rossiya/avtomobili?f=ASgBAgECAUTyCrCKAQFFxpoMHHsiZnJvbSI6NDUwMDAwLCJ0byI6MTAwMDAwMH0&p={}'
    url = templ.format(page) 

    return url

In [2]:
get_url (99)

'https://www.avito.ru/moskva/kvartiry/prodam/novostroyka-ASgBAQICAUSSA8YQAUDmBxSOUg?cd=1&f=ASgBAQECAUSSA8YQAUDmBxSOUgFFxpoMHnsiZnJvbSI6NDAwMDAwMCwidG8iOjM1MDAwMDAwfQ&p=99'

In [124]:
import csv
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup as bs
import requests

from time import sleep


def get_car(car): ## Extract car data for a single car
    
    try:
        car_brand = car.find('div', class_ = 'iva-item-titleStep-_CxvN').text
    except AttributeError:
        car_brand = ''
    try:
        car_price = car.find('span', class_ = 'price-price-BQkOZ').text.strip()
        car_price =  car_price.replace(u'\xa0', ' ')
    except AttributeError:
        car_price = ''
    try:
        url = car.div.a.get('href')
        car_url = 'https://www.avito.ru/' + url
    except AttributeError:
        car_url = ''
    try:
        car_details = car.find('div', class_ = 'iva-item-text-_s_vh text-text-LurtD text-size-s-BxGpL').text.strip()
    except AttributeError:
        car_details = ''
    try:
        car_location = car.find('div', class_ = 'geo-root-H3eWU iva-item-geo-g3iIJ').text.strip()
    except AttributeError:
        car_location = ''
    try:
        car_dealer = car.find('div',class_ = 'iva-item-text-_s_vh iva-item-hideWide-oLeJu text-text-LurtD text-size-s-BxGpL').text.strip()
    except AttributeError:
        car_dealer = ''
    try:
        car_description = car.find('div', class_ = 'iva-item-descriptionStep-QGE8Y').text.strip()
    except AttributeError:
        car_description = ''
        
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        ad_date = car.find('div', 
        class_ = 'date-text-VwmJG text-text-LurtD text-size-s-BxGpL text-color-noaccent-P1Rfs').text.strip()
    except AttributeError:
        ad_date = ''
        
    
    record = (car_brand,car_price,car_location, car_details, ad_date, car_dealer, car_description, today, car_url)
    
    return record
    

## Main working function    
def main():
    records = []
    headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
    
    #for page in range (0,101): ## Currently avito allows 100 pages max
    for page in range (0,101):
        url = get_url (page)
        r = requests.get(url, headers)

        if r: # A visual output durig scrapeing, otheriwise if else can be avoided
            print ('success getting r', page)
            soup = bs(r.content, 'html.parser')
            cars = soup.find_all('div', {'data-marker': 'item'})
            print ('found', ' ', len(cars), 'cars', ' ', 'from page', page)
            
            for car in cars:
                record = get_car(car)
                records.append(record)
            sleep (7)
        else:
            print ('Bad r', page)
       
            
    ## Save Data
    with open('file_name', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['car_brand','car_price','car_location', 'car_details', 'ad_date',
                         'car_dealer', 'car_description', 'today', 'car_url'])
        writer.writerows (records)
    
    

In [125]:
main()

success getting r 0
found   56 cars   from page 0
success getting r 1
found   56 cars   from page 1
success getting r 2
found   56 cars   from page 2
success getting r 3
found   56 cars   from page 3
success getting r 4
found   56 cars   from page 4
success getting r 5
found   56 cars   from page 5
success getting r 6
found   56 cars   from page 6
success getting r 7
found   56 cars   from page 7
success getting r 8
found   56 cars   from page 8
success getting r 9
found   56 cars   from page 9
success getting r 10
found   56 cars   from page 10
success getting r 11
found   56 cars   from page 11
success getting r 12
found   56 cars   from page 12
success getting r 13
found   56 cars   from page 13
success getting r 14
found   56 cars   from page 14
success getting r 15
found   56 cars   from page 15
success getting r 16
found   56 cars   from page 16
success getting r 17
found   56 cars   from page 17
success getting r 18
found   56 cars   from page 18
success getting r 19
found   56 

In [126]:
df = pd.read_csv('file_name.csv')

df.head(5)

,car_brand,car_price,car_location,car_details,ad_date,car_dealer,car_description,today,car_url
0,"Mitsubishi Lancer, 2014",650 000 ₽,Москва,"100 181 км, 1.8 CVT (140 л.с.), седан, передни...",Несколько секунд назад,NaN,Прoдаю надёжный автомобиль в достойнойном тeхн...,2021-11-23,https://www.avito.ru//moskva/avtomobili/mitsub...
1,"Mazda 3, 2008",490 000 ₽,"Свердловская область, Екатеринбург","213 000 км, 1.6 MT (105 л.с.), седан, передний...",Несколько секунд назад,NaN,"По кузову имеется один окрас, остальное все в ...",2021-11-23,https://www.avito.ru//ekaterinburg/avtomobili/...
2,"ВАЗ (LADA) Granta, 2020",742 000 ₽,"Свердловская область, Екатеринбург","25 600 км, 1.6 MT (106 л.с.), седан, передний,...",Несколько секунд назад,ИЮЛЬ авто с пробегом,= Самый большой Автомолл! Более 700 автомобиле...,2021-11-23,https://www.avito.ru//ekaterinburg/avtomobili/...
3,"Toyota Tank, 2018",590 000 ₽,"Приморский край, Владивостокский г.о., Владиво...","43 000 км, 1.0 CVT (69 л.с.), хетчбэк, полный,...",Несколько секунд назад,NaN,Автомобиль под заказ из Японии. \n\nПробег ука...,2021-11-23,https://www.avito.ru//vladivostok/avtomobili/t...
4,"Kia Rio, 2018",999 000 ₽,"Астраханская область, Астрахань","65 000 км, 1.6 AT (123 л.с.), седан, передний,...",Несколько секунд назад,NaN,"Автомобиль в отличном состоянии, вложений не т...",2021-11-23,https://www.avito.ru//astrahan/avtomobili/kia_...


In [127]:
df.shape

(5656, 9)